## Imports and Assigning Variable

In [63]:
import pandas as pd # vector junk
import json # json junk
import requests # handle requests
import folium # map rendering library
from bs4 import BeautifulSoup
import geopandas as gpd
from geopy.geocoders import Nominatim
print('Libraries imported.')

Libraries imported.


In [64]:
# Information created from the foursquare website to gain access to the api 
CLIENT_ID = 'X5K1NNIUOTZKXXKUNZYNGIOWPEK4WT4QU1PRDE2LEBOMMGF5'
CLIENT_SECRET = 'fsq3lMBZ7wsmrn8h8J1Rp/VHo5tAuxKiFoim2zid+FLT64g='
VERSION = 20220304
radius = 500
LIMIT = 100
print('Variables set') 

Variables set


## Collect Data and create DataFrames

Collect relevant data from sources and write into pandas data frames.

### Crime Data
Crime data sourced from: https://data.torontopolice.on.ca/datasets/TorontoPS::neighbourhood-crime-rates-boundary-file-/about

Contains Toronto Neigbhourhood crime data between 2014 and 2019. Provides counts on Assault, Auto Theft, Break and Enter, Robbery, Theft Over and Homicide per 100,000 people.

In [65]:
# Crime data original source from folder
crime_orig = pd.read_csv("Neighbourhoods/Neighbourhood_Crime_Rates.csv")
crime_orig.head()

,OBJECTID,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,...,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length
0,1,Yonge-St.Clair,97,12528,20,29,39,27,34,37,...,5,8,0,3,6,4.3,1.00,47.9,1.161315e+06,5873.270582
1,2,York University Heights,27,27593,271,296,361,344,357,370,...,46,37,39,38,28,36.3,-0.26,101.5,1.324666e+07,18504.777326
2,3,Lansing-Westgate,38,16164,44,80,68,85,75,72,...,5,5,11,6,11,7.0,0.83,68.1,5.346186e+06,11112.109625
3,4,Yorkdale-Glen Park,31,14804,106,136,174,161,175,209,...,14,26,23,20,29,22.5,0.45,195.9,6.038326e+06,10079.426920
4,5,Stonegate-Queensway,16,25051,88,71,76,95,87,82,...,8,4,6,7,4,6.0,-0.43,16.0,7.946202e+06,11853.189878


#### Create Modified Crime Dataframe

Convert the original crime dataframe into a modified version containing only
relevant data from latest crime statistics. Used when merging into combined
dataframe later on.

In [66]:
crime_df = crime_orig
# Assign variable name to list of relevant columns
crimes = ['Assault_2019', 'AutoTheft_2019', 'BreakandEnter_2019', 'Homicide_2019',
            'Robbery_2019', 'TheftOver_2019']
# Obtain Crime rate for each neighbourhood
crime_df['Crime_Rate'] = 100000 * (crime_df[crimes].sum(axis=1) /
                                     crime_df['Population'])
crime_df = pd.DataFrame(crime_df, columns=['Neighbourhood', 'Population', 'Assault_2019', 'AutoTheft_2019', 'BreakandEnter_2019', 'Homicide_2019',
            'Robbery_2019', 'TheftOver_2019', 'Crime_Rate'])
crime_df

,Neighbourhood,Population,Assault_2019,AutoTheft_2019,BreakandEnter_2019,Homicide_2019,Robbery_2019,TheftOver_2019,Crime_Rate
0,Yonge-St.Clair,12528,37,6,28,0,4,6,646.551724
1,York University Heights,27593,370,144,108,0,79,28,2641.974414
2,Lansing-Westgate,16164,72,32,39,0,11,11,1020.786934
3,Yorkdale-Glen Park,14804,209,61,84,1,42,29,2877.600648
4,Stonegate-Queensway,25051,82,34,64,0,22,4,822.322462
...,...,...,...,...,...,...,...,...,...
135,Milliken,26572,99,83,132,1,33,22,1392.443173
136,Pleasant View,15818,44,24,12,0,15,4,625.869263
137,Wychwood,14349,74,7,47,1,9,0,961.739494
138,Leaside-Bennington,16828,31,32,44,0,2,7,689.327312


### Neighbourhood Data

Collect data about for neighbourhood profiles and write into DataFrame.

Sourced from: https://open.toronto.ca/dataset/neighbourhoods/

In [67]:
# Neighbourhood Data

info_df_orig = pd.read_json('Neighbourhoods/neighbourhood-profiles-2016-csv.json')
# info_df_orig.rename(columns={'Neighbourhood':'Characteristic'})
# print(info_df_orig.columns)
info_df = info_df_orig.set_index('Characteristic').T

info_df


Characteristic,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Children (0-14 years),...,Intraprovincial migrants,Interprovincial migrants,External migrants,Total - Mobility status 5 years ago - 25% sample data,Non-movers,Movers,Non-migrants,Migrants,Interprovincial migrants,External migrants
Agincourt North,129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",7.41,"3,840",...,275,75,605,"27,490","18,865","8,610","5,445","3,170",135,"2,280"
Agincourt South-Malvern West,128,No Designation,"23,757","21,988",8.00%,"8,535","8,136","3,034",7.83,"3,075",...,320,90,490,"22,325","13,565","8,775","5,610","3,145",220,"2,170"
Alderwood,20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",4.95,"1,760",...,220,40,70,"11,370","8,235","3,130","2,200",925,70,245
Annex,95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",2.81,"2,360",...,900,385,835,"27,715","12,980","14,735","8,340","6,390","1,310","2,460"
Banbury-Don Mills,42,No Designation,"27,695","26,918",2.90%,"12,473","12,124","2,775",9.98,"3,605",...,345,75,380,"25,925","16,300","9,625","6,480","3,140",220,"1,735"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yonge-Eglinton,100,No Designation,"11,817","10,578",11.70%,"6,103","5,676","7,162",1.65,"1,800",...,345,130,325,"11,230","5,785","5,450","3,020","2,425",290,"1,160"
Yonge-St.Clair,97,No Designation,"12,528","11,652",7.50%,"7,475","7,012","10,708",1.17,"1,210",...,320,110,265,"11,865","5,660","6,195","3,895","2,310",325,955
York University Heights,27,NIA,"27,593","27,713",-0.40%,"11,051","10,170","2,086",13.23,"4,045",...,500,180,680,"26,110","14,720","11,400","6,435","4,965",195,"3,285"
Yorkdale-Glen Park,31,Emerging Neighbourhood,"14,804","14,687",0.80%,"5,847","5,344","2,451",6.04,"1,960",...,145,35,140,"13,420","9,125","4,290","2,940","1,345",135,775


### Merge Data

Merge crime and neighbourhood profile data into single dataframe.

In [68]:
merged = pd.merge(crime_df,info_df,left_on=['Neighbourhood'],right_index=True)
merged

,Neighbourhood,Population,Assault_2019,AutoTheft_2019,BreakandEnter_2019,Homicide_2019,Robbery_2019,TheftOver_2019,Crime_Rate,Neighbourhood Number,...,Intraprovincial migrants,Interprovincial migrants,External migrants,Total - Mobility status 5 years ago - 25% sample data,Non-movers,Movers,Non-migrants,Migrants,Interprovincial migrants,External migrants
0,Yonge-St.Clair,12528,37,6,28,0,4,6,646.551724,97,...,320,110,265,"11,865","5,660","6,195","3,895","2,310",325,955
1,York University Heights,27593,370,144,108,0,79,28,2641.974414,27,...,500,180,680,"26,110","14,720","11,400","6,435","4,965",195,"3,285"
2,Lansing-Westgate,16164,72,32,39,0,11,11,1020.786934,38,...,440,110,510,"15,275","7,930","7,335","4,170","3,165",220,"1,965"
3,Yorkdale-Glen Park,14804,209,61,84,1,42,29,2877.600648,31,...,145,35,140,"13,420","9,125","4,290","2,940","1,345",135,775
4,Stonegate-Queensway,25051,82,34,64,0,22,4,822.322462,16,...,480,150,225,"23,510","16,000","7,515","4,780","2,730",250,"1,245"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Milliken,26572,99,83,132,1,33,22,1392.443173,130,...,290,10,500,"24,730","16,725","8,000","5,105","2,895",85,"2,120"
136,Pleasant View,15818,44,24,12,0,15,4,625.869263,46,...,250,75,575,"15,255","9,910","5,345","2,865","2,475",110,"1,765"
137,Wychwood,14349,74,7,47,1,9,0,961.739494,94,...,140,60,225,"12,630","7,930","4,700","3,355","1,350",210,575
138,Leaside-Bennington,16828,31,32,44,0,2,7,689.327312,56,...,125,85,190,"15,845","11,135","4,710","3,560","1,145",200,380


### Location Data

Information regarding location data for neighbourhoods. Includes Lattitude, Longitude and neighbour shape (geometry)

Sourced from: https://open.toronto.ca/dataset/neighbourhoods/

In [69]:
#Get Latitude and Longitude from Neighbourhoods File

#Read the shape file to get longitude and latitude when crs epsg=4326
LocationData = gpd.read_file("Neighbourhoods/Neighbourhoods.shp")

#Converted to get Latitude and Longitude
LocationData = LocationData.to_crs(epsg=4326)

#Enter data into a dataframe
NeighbourhoodLocationData = pd.DataFrame(LocationData,columns=['FIELD_7', 'FIELD_12', 'FIELD_11', 'geometry'])
#NeighbourhoodLocationData = pd.DataFrame(LocationData)
NeighbourhoodLocationData
# #Rename all columns
NeighbourhoodLocationData.rename(columns={'FIELD_7':'Neighbourhood', 'FIELD_12':'Latitude', 'FIELD_11':'Longitude', 'geometry':'geometry'}, inplace=True)

# #Get all the values before the bracket
NeighbourhoodLocationData['Neighbourhood'] = NeighbourhoodLocationData["Neighbourhood"].astype(str)
NeighbourhoodLocationData['Neighbourhood'] = NeighbourhoodLocationData['Neighbourhood'].str.split(" \(", 1)
NeighbourhoodLocationData['Neighbourhood'] = NeighbourhoodLocationData['Neighbourhood'].str.get(0)

NeighbourhoodLocationData.head()
#print(NeighbourhoodLocationData['Neighbourhood'].value_counts())
#NeighbourhoodLocationData.to_csv("outputL.csv")

C:\Users\moham\anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Neighbourhood,Latitude,Longitude,geometry
0,Wychwood,43.676919,-79.425515,"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
1,Yonge-Eglinton,43.704689,-79.403590,"POLYGON ((-79.41096 43.70408, -79.40962 43.704..."
2,Yonge-St.Clair,43.687859,-79.397871,"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."
3,York University Heights,43.765736,-79.488883,"POLYGON ((-79.50529 43.75987, -79.50488 43.759..."
4,Yorkdale-Glen Park,43.714672,-79.457108,"POLYGON ((-79.43969 43.70561, -79.44011 43.705..."


### Combine all data sets

Combine the merged neighbourhood profiles and crime rates with the location data.

Select relevant columns and write to new combined data frame

In [70]:
#combination of all the datasets

#Step 1 merge information on Neighbourhood location data and Neighbourhood profiles
F1 = pd.merge(merged, NeighbourhoodLocationData, on="Neighbourhood")
# F1

#F1.to_csv("outputF1.csv")


#Step 2 merge information on Neighbourhood postal code data and previous merge
# F2 = pd.merge(F1, MergeLocation, on="Neighbourhood")
# F2
F2 = F1[['Neighbourhood', 'Population','Crime_Rate', 'Youth (15-24 years)', 'Working Age (25-54 years)', 'Latitude', 'Longitude', 'geometry']]
F2.head()


,Neighbourhood,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Latitude,Longitude,geometry
0,Yonge-St.Clair,12528,646.551724,920,"5,960",43.687859,-79.397871,"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."
1,York University Heights,27593,2641.974414,"4,750","12,290",43.765736,-79.488883,"POLYGON ((-79.50529 43.75987, -79.50488 43.759..."
2,Lansing-Westgate,16164,1020.786934,"2,105","7,590",43.754271,-79.424748,"POLYGON ((-79.43998 43.76156, -79.44004 43.761..."
3,Yorkdale-Glen Park,14804,2877.600648,"1,870","5,860",43.714672,-79.457108,"POLYGON ((-79.43969 43.70561, -79.44011 43.705..."
4,Stonegate-Queensway,25051,822.322462,"2,495","10,660",43.635518,-79.501128,"POLYGON ((-79.49262 43.64744, -79.49277 43.647..."


## Collecting Venue Information from the Foursquare API
After creating a foursquare account at https://foursquare.com/developers/signup the following algorithm was built
Using category from https://developer.foursquare.com/docs/categories

In [71]:
# Variable sets
radius = 2000 # Perimeter for venues
limit = 50 # Maximum amount of venues per Neighbourhood
category = 13064 # category Dining and Drinking > Restaurant > Pizzeria


places = [] # data for the frame 

# Taking geographic coordinates from the NeighbourLocationData Frame
for lat, long, neighbourhood in zip(NeighbourhoodLocationData['Latitude'],
                                   NeighbourhoodLocationData['Longitude'],
                                   NeighbourhoodLocationData['Neighbourhood']):
    
    url = "https://api.foursquare.com/v3/places/search?ll={}%2C{}&radius={}&categories={}&limit={}".format(lat,long,radius,category,limit)
   
    headers = {

        "Accept": "application/json",

        "Authorization": "fsq346wopDU+Ro11RNJhAnUwpiz/2xgtt3pjAunkFmd1c+8="

    }

    # Response from the website
    response = requests.request("GET", url, headers=headers).json()["results"]
    
    # building the places data set                                                               
    for place in response:
        places.append((neighbourhood,
                      lat,
                      long,
                      place['name'],
                      place['geocodes']['main']['latitude'],
                      place['geocodes']['main']['longitude'],
                     ))
        
print("Loaded to list!") # notifying that all venues have been loaded in to the dataset


Loaded to list!


### Creating a pandas data frame leveraging the places dataset
from the foursquare APIs venue data a pandas data frame is built 

In [72]:
places_df = pd.DataFrame(places)
# pd.set_option("display.max_rows", None)

places_df.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'Name', 'VenueLatitude', 'VenueLongitude']
places_df



,Neighbourhood,Latitude,Longitude,Name,VenueLatitude,VenueLongitude
0,Wychwood,43.676919,-79.425515,Ferro Bar & Cafe,43.680943,-79.428443
1,Wychwood,43.676919,-79.425515,Marcello's Pizzeria,43.677841,-79.442494
2,Wychwood,43.676919,-79.425515,Churrasco of St Clair,43.681630,-79.425185
3,Wychwood,43.676919,-79.425515,Napoli Centrale,43.669377,-79.413027
4,Wychwood,43.676919,-79.425515,Pizza e Pazzi,43.677954,-79.444009
...,...,...,...,...,...,...
3751,L'Amoreaux,43.795716,-79.314084,Chef's Pizza Fish & Chips,43.795980,-79.327180
3752,L'Amoreaux,43.795716,-79.314084,Angela Pizza & Wings,43.787750,-79.329066
3753,L'Amoreaux,43.795716,-79.314084,Sammy's Pizza & Restaurant,43.786489,-79.328632
3754,L'Amoreaux,43.795716,-79.314084,Pizza 3 Toppings,43.783493,-79.299138


### Counting the total venues for pizza locations
Using the places data frame the total venues for pizza shops for each neighbourhood was calculated and made into a frame

In [73]:
# pd.set_option("display.max_rows", None)

places_df
count = pd.DataFrame(places_df['Neighbourhood'].value_counts().reset_index())
count = count.rename(columns={'index':'Neighbourhood', 'Neighbourhood':'Total Pizzerias'})
count



,Neighbourhood,Total Pizzerias
0,Wychwood,50
1,Trinity-Bellwoods,50
2,North St.James Town,50
3,Oakwood Village,50
4,Palmerston-Little Italy,50
...,...,...
135,Edenbridge-Humber Valley,4
136,Bayview Village,3
137,Princess-Rosethorn,3
138,Bayview Woods-Steeles,2


### Merging the Neighbourhood information data frame with the venue data
Merges the Neighbourhood data set with the total venue dataset

In [74]:
#Frame of all relevent information plus venue information
F3 = pd.merge(F2, count, on="Neighbourhood")
F3.head()

,Neighbourhood,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Latitude,Longitude,geometry,Total Pizzerias
0,Yonge-St.Clair,12528,646.551724,920,"5,960",43.687859,-79.397871,"POLYGON ((-79.39119 43.68108, -79.39141 43.680...",30
1,York University Heights,27593,2641.974414,"4,750","12,290",43.765736,-79.488883,"POLYGON ((-79.50529 43.75987, -79.50488 43.759...",21
2,Lansing-Westgate,16164,1020.786934,"2,105","7,590",43.754271,-79.424748,"POLYGON ((-79.43998 43.76156, -79.44004 43.761...",18
3,Yorkdale-Glen Park,14804,2877.600648,"1,870","5,860",43.714672,-79.457108,"POLYGON ((-79.43969 43.70561, -79.44011 43.705...",26
4,Stonegate-Queensway,25051,822.322462,"2,495","10,660",43.635518,-79.501128,"POLYGON ((-79.49262 43.64744, -79.49277 43.647...",17


#### Creating a temporary frame for geometry
for folium map

In [75]:
Ftemp = pd.DataFrame(F3, columns=['geometry'])

#### Removing commas from data
For the clusters data has to be in integer format

In [76]:
#Dataset without commas in ints
F4 = pd.DataFrame(F3,columns=['Neighbourhood', 'Population','Crime_Rate', 'Youth (15-24 years)', 'Working Age (25-54 years)', 'Total Pizzerias', 'Latitude', 'Longitude'])

F4['Youth (15-24 years)'] = pd.to_numeric(F3['Youth (15-24 years)'].str.replace(",",""))
F4['Working Age (25-54 years)'] = pd.to_numeric(F3['Working Age (25-54 years)'].str.replace(",",""))

F4

,Neighbourhood,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias,Latitude,Longitude
0,Yonge-St.Clair,12528,646.551724,920,5960,30,43.687859,-79.397871
1,York University Heights,27593,2641.974414,4750,12290,21,43.765736,-79.488883
2,Lansing-Westgate,16164,1020.786934,2105,7590,18,43.754271,-79.424748
3,Yorkdale-Glen Park,14804,2877.600648,1870,5860,26,43.714672,-79.457108
4,Stonegate-Queensway,25051,822.322462,2495,10660,17,43.635518,-79.501128
...,...,...,...,...,...,...,...,...
131,Milliken,26572,1392.443173,3195,10565,11,43.820691,-79.275009
132,Pleasant View,15818,625.869263,2080,6470,22,43.786982,-79.334948
133,Wychwood,14349,961.739494,1320,6420,50,43.676919,-79.425515
134,Leaside-Bennington,16828,689.327312,2175,6455,24,43.703797,-79.366072


## Clustering 
Using the final Neighbourhood data frame clustering begins using kmeans

In [77]:
from sklearn.cluster import KMeans
Cluster_data=F4.copy(deep=True) # make a new dataframe for clustering
# Drop columns not required for the clustering algorithm
ClusteringPer = Cluster_data.drop(labels=['Neighbourhood'], axis=1)

# set number of clusters
k = 4

# run k-means clustering
kmean_algo = KMeans(n_clusters=k, random_state=0).fit(ClusteringPer)

# check cluster labels generated for each row in the dataframe
kmean_algo.labels_[0:7] 

# add clustering labels to original dataframe
Cluster_data.insert(1, 'Clusters', kmean_algo.labels_)
#Cluster_data.to_csv('CleanData/Clusters.csv')
# print(kmeans.labels_)

Cluster_data = pd.concat([Cluster_data, Ftemp], join='inner', axis=1)
Cluster_data.head()

,Neighbourhood,Clusters,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias,Latitude,Longitude,geometry
0,Yonge-St.Clair,3,12528,646.551724,920,5960,30,43.687859,-79.397871,"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."
1,York University Heights,2,27593,2641.974414,4750,12290,21,43.765736,-79.488883,"POLYGON ((-79.50529 43.75987, -79.50488 43.759..."
2,Lansing-Westgate,0,16164,1020.786934,2105,7590,18,43.754271,-79.424748,"POLYGON ((-79.43998 43.76156, -79.44004 43.761..."
3,Yorkdale-Glen Park,3,14804,2877.600648,1870,5860,26,43.714672,-79.457108,"POLYGON ((-79.43969 43.70561, -79.44011 43.705..."
4,Stonegate-Queensway,2,25051,822.322462,2495,10660,17,43.635518,-79.501128,"POLYGON ((-79.49262 43.64744, -79.49277 43.647..."


### Average of all clusters
Finding the average of each clusters to narrow down the neighbourhoods 

In [78]:
averageOfClusters = Cluster_data.groupby('Clusters').mean()
averageOfClusters = pd.DataFrame(averageOfClusters, columns=['Population', 'Crime_Rate', 'Youth (15-24 years)', 'Working Age (25-54 years)', 'Total Pizzerias'])
averageOfClusters

,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias
Clusters,,,,,
0,19216.693878,1325.717100,2356.530612,8410.204082,24.265306
1,49725.714286,1183.646089,6597.857143,23875.000000,19.714286
2,29346.680000,1771.016378,3871.600000,13465.800000,24.080000
3,11506.545455,1238.754192,1348.363636,4994.000000,30.127273


## Results

In [79]:
# Cluster 1 seems like the best fit when examining the business case

### Spliting clusters into data frames

In [80]:
# Assigning the clusters to their own dataframes to eventually plot
#Cluster_data
is_0 = Cluster_data['Clusters'] == 0
Cluster0 = Cluster_data[is_0]

is_1 = Cluster_data['Clusters'] == 1
Cluster1 = Cluster_data[is_1]

is_2 = Cluster_data['Clusters'] == 2
Cluster2 = Cluster_data[is_2]

is_3 = Cluster_data['Clusters'] == 3
Cluster3 = Cluster_data[is_3]

In [81]:
#Cluster0 ## 2nd ranked data set
#Cluster1 ## Best Dataset 1st ranked data set
#Cluster2 ## 3rd ranked data set
#Cluster3 ## worst data set 4th ranked data set

### Mapping
Using the geometry the clusters were mapped according to their ranking:
lowest to highest is (3,2,0,1) 
Cluster3: worst fit, Cluster2: Moderately Fit, Cluster0: Near Fit, Cluster1: Ideal Fit
and the colours that represent them are red, cyan, yellow, and green
Cluster3: red, Cluster2: cyan, Cluster0: yellow, Cluster1: green

In [82]:
#Adding filling on the map for locations

map = folium.Map(location=[43.65, - 79.38], zoom_start=100)
# adding colour of gold
for x, r in Cluster0.iterrows():
    simulation = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geometryinfo = simulation.to_json()
    geometryinfo = folium.GeoJson(data=geometryinfo,
                           style_function=lambda x: {'fillColor': 'Yellow'})
    folium.Popup(r['Neighbourhood']).add_to(geometryinfo)
    geometryinfo.add_to(map)

for x, r in Cluster1.iterrows():
    simulation = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geometryinfo = simulation.to_json()
    geometryinfo = folium.GeoJson(data=geometryinfo,
                           style_function=lambda x: {'fillColor': 'Green'})
    folium.Popup(r['Neighbourhood']).add_to(geometryinfo)
    geometryinfo.add_to(map)

for x, r in Cluster2.iterrows():
    simulation = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geometryinfo = simulation.to_json()
    geometryinfo = folium.GeoJson(data=geometryinfo,
                           style_function=lambda x: {'fillColor': 'Cyan'})
    folium.Popup(r['Neighbourhood']).add_to(geometryinfo)
    geometryinfo.add_to(map)

for x, r in Cluster3.iterrows():
    simulation = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geometryinfo = simulation.to_json()
    geometryinfo = folium.GeoJson(data=geometryinfo,
                           style_function=lambda x: {'fillColor': 'Red'})
    folium.Popup(r['Neighbourhood']).add_to(geometryinfo)
    geometryinfo.add_to(map)

map

### Data Set for Ideal fit Cluster

In [83]:
# Best data set represents green on the folium map above
Cluster1

,Neighbourhood,Clusters,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias,Latitude,Longitude,geometry
11,Islington-City Centre West,1,43965,1344.251109,4695,20640,18,43.633463,-79.543317,"POLYGON ((-79.51317 43.62049, -79.51505 43.620..."
39,Waterfront Communities-The Island,1,65913,2139.183469,7840,45105,50,43.633880,-79.377202,"POLYGON ((-79.37697 43.64688, -79.37576 43.647..."
82,L'Amoreaux,1,43993,861.500693,5730,17210,11,43.795716,-79.314084,"POLYGON ((-79.28857 43.79607, -79.28921 43.795..."
93,Willowdale East,1,50434,747.511599,6940,25850,29,43.770602,-79.401484,"POLYGON ((-79.41225 43.76669, -79.41232 43.766..."
95,Rouge,1,46496,802.219546,6700,18510,2,43.821201,-79.186343,"POLYGON ((-79.19701 43.79652, -79.19705 43.796..."
108,Woburn,1,53485,1237.730205,7660,21945,20,43.766740,-79.228586,"POLYGON ((-79.21128 43.75113, -79.21135 43.751..."
119,Malvern,1,43794,1153.125999,6620,17865,8,43.803658,-79.222517,"POLYGON ((-79.19701 43.79652, -79.19744 43.796..."


#### Adding a function to calculate Desirablity 

In [84]:
Final_Params = Cluster1
Final_Params['Ideal Factor'] = (Cluster1['Youth (15-24 years)']+Cluster1['Working Age (25-54 years)'])/(Cluster1['Total Pizzerias'] * Cluster1['Crime_Rate'])

Final_Params

C:\Users\moham\AppData\Local\Temp/ipykernel_16972/323256444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Params['Ideal Factor'] = (Cluster1['Youth (15-24 years)']+Cluster1['Working Age (25-54 years)'])/(Cluster1['Total Pizzerias'] * Cluster1['Crime_Rate'])


,Neighbourhood,Clusters,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias,Latitude,Longitude,geometry,Ideal Factor
11,Islington-City Centre West,1,43965,1344.251109,4695,20640,18,43.633463,-79.543317,"POLYGON ((-79.51317 43.62049, -79.51505 43.620...",1.047051
39,Waterfront Communities-The Island,1,65913,2139.183469,7840,45105,50,43.633880,-79.377202,"POLYGON ((-79.37697 43.64688, -79.37576 43.647...",0.495002
82,L'Amoreaux,1,43993,861.500693,5730,17210,11,43.795716,-79.314084,"POLYGON ((-79.28857 43.79607, -79.28921 43.795...",2.420723
93,Willowdale East,1,50434,747.511599,6940,25850,29,43.770602,-79.401484,"POLYGON ((-79.41225 43.76669, -79.41232 43.766...",1.512605
95,Rouge,1,46496,802.219546,6700,18510,2,43.821201,-79.186343,"POLYGON ((-79.19701 43.79652, -79.19705 43.796...",15.712656
108,Woburn,1,53485,1237.730205,7660,21945,20,43.766740,-79.228586,"POLYGON ((-79.21128 43.75113, -79.21135 43.751...",1.195939
119,Malvern,1,43794,1153.125999,6620,17865,8,43.803658,-79.222517,"POLYGON ((-79.19701 43.79652, -79.19744 43.796...",2.654198


#### Getting the Best Location

In [85]:
best_value = Final_Params['Ideal Factor'].max()

is_best = Cluster1['Ideal Factor'] == best_value
Prime_Location = Cluster1[is_best]
Prime_Location

,Neighbourhood,Clusters,Population,Crime_Rate,Youth (15-24 years),Working Age (25-54 years),Total Pizzerias,Latitude,Longitude,geometry,Ideal Factor
95,Rouge,1,46496,802.219546,6700,18510,2,43.821201,-79.186343,"POLYGON ((-79.19701 43.79652, -79.19705 43.796...",15.712656


#### Adding a Marker for the Best Location

In [86]:
for i in range(0,len(Prime_Location)):
       folium.Marker(
      location=[Prime_Location.iloc[i]['Latitude'], Prime_Location.iloc[i]['Longitude']],
      popup=Prime_Location.iloc[i]['Population'],
   ).add_to(map)
map
